# My little database JSON-LD import

## 1 Import relevant packages

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing
import bw_migrations
from bw2io.importers.json_ld_lcia import JSONLDLCIAImporter

import os
import numpy as np
import pandas as pd

## 2 Create/set the working folder

In [2]:
bd.projects.dir

PosixPath('/Users/mmendez/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682')

In [3]:
bd.projects.set_current('MLD_JSON_001') # Set the project name you want!

## 3 Import JSON-LD LCI

In [4]:
path = '../7_BW2_Importing_USLCI/databases/My_little_database' # For some reason if I use the relative path it throws me an error
mld = bi.importers.JSONLDImporter(
    path, 
    "My_little_dataset_v0", 
    preferred_allocation="PHYSICAL_ALLOCATION"
)

### 3.1 Apply strategies to map JSON-LD to Brightway2 schema

In [5]:
mld.apply_strategies()

Applying strategy: json_ld_allocate_datasets
Applying strategy: json_ld_get_normalized_exchange_locations
Applying strategy: json_ld_convert_unit_to_reference_unit
Applying strategy: json_ld_get_activities_list_from_rawdata
Applying strategy: json_ld_add_products_as_activities
Applying strategy: json_ld_get_normalized_exchange_units
Applying strategy: json_ld_add_activity_unit
Applying strategy: json_ld_rename_metadata_fields
Applying strategy: json_ld_location_name
Applying strategy: json_ld_remove_fields
Applying strategy: json_ld_fix_process_type
Applying strategy: json_ld_label_exchange_type
Applying strategy: json_ld_prepare_exchange_fields_for_linking
Applying strategy: add_database_name
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: normalize_units
Applied 17 strategies in 0.00 seconds

	Created 3 biosphere flows in separate database 'My_little_dataset_v0 biosphere'.
	Use either `.merge_biosphere_flows()` or `.write_separ

### 3.2 Check database dictionaries

In [6]:
bd.databases # As expected, nothing was written.

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

### 3.3 Write the biosphere database

In [7]:
mld.write_separate_biosphere_database()

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/22/2022 14:46:20
  Finished: 09/22/2022 14:46:20
  Total time elapsed: 00:00:00
  CPU %: 2.20
  Memory %: 0.49
Created database: My_little_dataset_v0 biosphere


Check that the database was written in the database dictionary.

In [8]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

Save the biosphere database in a variable for easy use.

In [9]:
bio = bd.Database('My_little_dataset_v0 biosphere')

### 3.4 Write the technosphere database

In [10]:
mld.write_database()

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/22/2022 14:46:20
  Finished: 09/22/2022 14:46:20
  Total time elapsed: 00:00:00
  CPU %: 2.60
  Memory %: 0.50
Created database: My_little_dataset_v0


Brightway2 SQLiteBackend: My_little_dataset_v0

<div class="alert alert-block alert-warning">
<b>To fix:</b>
    <li>The importer does not recognize the geolocations.</li>
</div>

Check that the database was written in the database dictionary.

In [11]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

In [12]:
db = bd.Database('My_little_dataset_v0')

## 4 Import the LCIA methods

In [13]:
mld_methods = bi.importers.JSONLDLCIAImporter(path)

In [14]:
mld_methods.apply_strategies()

Applying strategy: json_ld_lcia_add_method_metadata
Applying strategy: json_ld_lcia_convert_to_list
Applying strategy: json_ld_lcia_set_method_metadata
Applying strategy: json_ld_lcia_reformat_cfs_as_exchanges
Applying strategy: normalize_units
Applied 5 strategies in 0.00 seconds


In [15]:
mld_methods.match_biosphere_by_id('My_little_dataset_v0 biosphere')

In [16]:
mld_methods.statistics()

1 methods
1 cfs
0 unlinked cfs


(1, 1, 0)

In [17]:
mld_methods.data

dict_values([{'name': ('Mac IAM', 'Global warming'), 'version': '00.00.000', 'filename': '/Users/mmendez/Documents/Postdoc/Software_dev/Brightway/BW_Tutorials/learningbrightway/7_BW2_Importing_USLCI/databases/My_little_database/lcia_categories/6ff74fb6-3d8e-4c2e-ba7e-64e2fd4376c4.json', 'parent': {'name': 'Mac IAM', 'description': '', 'version': '00.00.001'}, 'unit': 'kg CO2 eq', 'id': '6ff74fb6-3d8e-4c2e-ba7e-64e2fd4376c4', 'description': '\n', 'exchanges': [{'@type': 'ImpactFactor', 'flow': {'@type': 'Flow', '@id': 'cd154d8f-0694-43b2-b4ab-e44101e122bd', 'name': 'Bad stuff', 'categoryPath': ['Elementary flows', 'emissions'], 'flowType': 'ELEMENTARY_FLOW', 'location': 'DJ', 'refUnit': 'kg'}, 'unit': 'kilogram', 'flowProperty': {'@type': 'FlowProperty', '@id': '65d0b9d1-3295-4e2d-8b1c-71ef05c71fe2', 'name': 'Mass'}, 'amount': 2.0, 'input': ('My_little_dataset_v0 biosphere', 'cd154d8f-0694-43b2-b4ab-e44101e122bd')}]}])

In [18]:
[things['name'] for things in mld_methods.data]

[('Mac IAM', 'Global warming')]

In [19]:
if [things['name'] for things in mld_methods.data][0] in bd.methods:
    print('Method already loaded!')
else:
    mld_methods.write_methods()

Method already loaded!


In [20]:
mld_methods

In [21]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

## 5 Activity querying

### 5.1 Check the biosphere activities

In [22]:
[act for act in bio]

['Virgin metals' (, None, ('Elementary flows', 'resources')),
 'Crude oil' (, None, ('Elementary flows', 'resources')),
 'Bad stuff' (, None, ('Elementary flows', 'emissions'))]

### 5.2 And now let's look at the our processes and products:

In [23]:
[(act.as_dict(), act['unit']) for act in bio]

[({'code': '7528830a-7344-43be-b484-ac4dc625f272',
   'name': 'Virgin metals',
   'categories': ('Elementary flows', 'resources'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 19},
  ''),
 ({'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
   'name': 'Bad stuff',
   'categories': ('Elementary flows', 'emissions'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 21},
  ''),
 ({'code': '8849be54-1b13-4d7e-85f6-2297817333f2',
   'name': 'Crude oil',
   'categories': ('Elementary flows', 'resources'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 20},
  '')]

In [24]:
[(act, act['type'], act['unit']) for act in db]

[('Plastic manufacturing' (kilogram, Djibouti, None), 'process', 'kilogram'),
 ('Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')), 'product', ''),
 ('Stainless steel manufacturing' (kilogram, Djibouti, None),
  'process',
  'kilogram'),
 ('Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
  'product',
  ''),
 ('Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
  'product',
  ''),
 ('Bottle assembly' (number_of_items, Djibouti, None),
  'process',
  'number_of_items')]

In [25]:
[(act['name'],[exc for exc in act.exchanges()]) for act in db if act['type'] == 'process']

[('Stainless steel manufacturing',
  [Exchange: 1.0  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Virgin metals' (, None, ('Elementary flows', 'resources')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 3.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>]),
 ('Bottle assembly',
  [Exchange: 1.0  'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.25  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.1  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.3  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle a

In [26]:
[(act['name'],[exc for exc in act.exchanges()]) for act in db if act['type'] == 'process']

[('Plastic manufacturing',
  [Exchange: 1.0  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 2.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Crude oil' (, None, ('Elementary flows', 'resources')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>]),
 ('Bottle assembly',
  [Exchange: 1.0  'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.25  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.1  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.3  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>]

### 5.3 Which ones are processes and which ones are products? Let's also get their codes:

In [27]:
[(act['name'], act['type'], act['code']) for act in db]

[('Bottle assembly', 'process', 'fae6799b-7326-452c-92b3-76758bbcac22'),
 ('Bottle', 'product', 'b806c2cd-d563-43c4-a0c9-9c7dd5d513d3'),
 ('Stainless steel', 'product', '3355a1b9-8fbf-40de-b449-ea6399a8a323'),
 ('Stainless steel manufacturing',
  'process',
  '7e5ec332-09fd-4706-8373-3f140a539028'),
 ('Plastic manufacturing', 'process', '2fc6deea-6437-4b9f-bf91-89bca44d30f0'),
 ('Plastic', 'product', '62714200-1a0d-43fb-9b48-99df3f233c94')]

### 5.4 Exploring the LCIA methods

#### 5.4.1 How to manually add a LCIA method
Before I figured out how to add LCIA methods I wrote them manually, this is how it is done (uncomment if you want to run this).

In [28]:
bad_flow = [act for act in bio if act['name'] == 'Bad stuff'][0]
bad_flow.as_dict()

{'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
 'name': 'Bad stuff',
 'categories': ('Elementary flows', 'emissions'),
 'CAS number': None,
 'database': 'My_little_dataset_v0 biosphere',
 'unit': '',
 'type': 'emission',
 'id': 21}

In [29]:
myLCIAdata = [[(bad_flow['database'], bad_flow['code']), 2.0]] # A method list needs: a reference to the flow: tuple (database, 'code')), a characterization factor number, and localization (if no localization is given, 'GLO' is used)
method_key_manual = ('MacIM_manual', 'Global warming')
my_method_manual = bd.Method(method_key_manual)
my_method_manual.validate(myLCIAdata)
my_method_manual.register()
my_method_manual.write(myLCIAdata)

#### 5.4.2 Search through our JSON-LD imported methods

In [30]:
bd.methods

Methods dictionary with 2 object(s):
	('Mac IAM', 'Global warming')
	('MacIM_manual', 'Global warming')

Search works the same as before, we just need to use line comprehension as before.

In [31]:
[method for method in bd.methods]

[('Mac IAM', 'Global warming'), ('MacIM_manual', 'Global warming')]

In [32]:
method_key = [method for method in bd.methods][0] # [0] selects the method
method_key

('Mac IAM', 'Global warming')

In [33]:
my_method = bd.Method(method_key)

In [34]:
my_method.name

('Mac IAM', 'Global warming')

In [35]:
my_method.metadata['unit']

'kg CO2 eq'

In [36]:
my_method.load()

[(('My_little_dataset_v0 biosphere', 'cd154d8f-0694-43b2-b4ab-e44101e122bd'),
  2.0)]

## 6 Running the LCA!

### 6.1 Define the functional unit:
Our functional unit here is **Impact of assembling 5 bottles**, so we need to select the flow coming out of the `Bottle assembly` activity, i.e. `Bottle`, which is a `product` not a `process`.

In [37]:
activity = [act for act in db if act['type'] == 'product' and act['name'] == 'Bottle'][0]
activity

'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers'))

In [38]:
functional_unit = {activity : 50} #Impact of 5 bottles

### 6.2 Run the LCA!

Let's run the LCA with the imported and manually created LCIA method.

In [39]:
lca = bc.LCA(functional_unit, method_key) 

In [40]:
lca_manual = bc.LCA(functional_unit, method_key_manual) 

In [41]:
print(len(dir(lca)))
print(len(dir(lca_manual)))

72
72


In [42]:
lca.lci()   # Builds matrices, solves the system, generates an LCI matrix.
lca_manual.lci()

In [43]:
print(len(dir(lca)))
print(len(dir(lca_manual)))

79
79


In [44]:
lca.lcia()  # Characterization, i.e. the multiplication of the elements  
            # of the LCI matrix with characterization factors from the chosen method
lca_manual.lcia() 

In [45]:
print(len(dir(lca)))
print(len(dir(lca_manual)))

82
82


In [46]:
print(lca.score )   # Returns the score, i.e. the sum of the characterized inventory
print(lca_manual.score)

0.0
150.0000037252903


BEAUTIFUL!

There is a bug though, if you re-run this code in the same project folder, the lca.score will be 0. I don't know why jet!